# Homework 2

In [98]:
# These are standard python modules
import json, time, urllib.parse, csv, pandas

# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3
import requests

### Read input politicians file

In [99]:
politicians_by_country = []
with open('politicians_by_country_SEPT.2022.csv', encoding='utf-8', mode ='r') as f:
    reader = csv.reader(f, delimiter=',')
    i = 0
    for row in reader:
        if i == 0:
            i = i + 1
        else:
            politicians_by_country.append(row)
len(politicians_by_country)

7584

### Common Utilities

In [100]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<uwnetid@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2022',
}

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}

In [101]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    # Make sure we have an article title
    if not article_title: return None
    
    request_template['titles'] = article_title
        
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

### Query the politician article's revision ID

In [106]:
i = 0
politician_with_revid = []
for politician in politicians_by_country:
    # print("Getting page info data for: ", politician[0])
    info = request_pageinfo_per_article(politician[0])
    pages = []
    for key in info['query']['pages']:
        pages.append(key)
    page = info['query']['pages'][pages[0]]
    if 'lastrevid' in page:
        revid = page['lastrevid']
        politician.append(revid)
        politician_with_revid.append(politician)
        i = i + 1
        if i % 100 == 0:
            print(i)
    else:
        print(page)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
{'ns': 0, 'title': 'Prince Ofosu Sefah', 'missing': '', 'contentmodel': 'wikitext', 'pagelanguage': 'en', 'pagelanguagehtmlcode': 'en', 'pagelanguagedir': 'ltr', 'fullurl': 'https://en.wikipedia.org/wiki/Prince_Ofosu_Sefah', 'editurl': 'https://en.wikipedia.org/w/index.php?title=Prince_Ofosu_Sefah&action=edit', 'canonicalurl': 'https://en.wikipedia.org/wiki/Prince_Ofosu_Sefah'}
2500
2600
2700
2800
2900
{'ns': 0, 'title': 'Harjit Kaur Talwandi', 'missing': '', 'contentmodel': 'wikitext', 'pagelanguage': 'en', 'pagelanguagehtmlcode': 'en', 'pagelanguagedir': 'ltr', 'fullurl': 'https://en.wikipedia.org/wiki/Harjit_Kaur_Talwandi', 'editurl': 'https://en.wikipedia.org/w/index.php?title=Harjit_Kaur_Talwandi&action=edit', 'canonicalurl': 'https://en.wikipedia.org/wiki/Harjit_Kaur_Talwandi'}
3000
3100
3200
{'ns': 0, 'title': 'Abd al-Razzaq al-Hasani', 'missing': '', 'contentmodel': 'w

In [107]:
#########
#
#    CONSTANTS
#

# The current ORES API endpoint
API_ORES_SCORE_ENDPOINT = "https://ores.wikimedia.org/v3"
# A template for mapping to the URL
API_ORES_SCORE_PARAMS = "/scores/{context}/{revid}/{model}"

# Use some delays so that we do not hammer the API with our requests
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<uwnetid@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2022'
}

# This template lists the basic parameters for making an ORES request
ORES_PARAMS_TEMPLATE = {
    "context": "enwiki",        # which WMF project for the specified revid
    "revid" : "",               # the revision to be scored - this will probably change each call
    "model": "articlequality"   # the AI/ML scoring model to apply to the reviewion
}
#
# The current ML models for English wikipedia are:
#   "articlequality"
#   "articletopic"
#   "damaging"
#   "version"
#   "draftquality"
#   "drafttopic"
#   "goodfaith"
#   "wp10"
#
# The specific documentation on these is scattered so if you want to use one you'll have to look around.
#

In [108]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, 
                                   endpoint_url = API_ORES_SCORE_ENDPOINT, 
                                   endpoint_params = API_ORES_SCORE_PARAMS, 
                                   request_template = ORES_PARAMS_TEMPLATE,
                                   headers = REQUEST_HEADERS,
                                   features=False):
    # Make sure we have an article revision id
    if not article_revid: return None
    
    # set the revision id into the template
    request_template['revid'] = article_revid
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # the features used by the ML model can sometimes be returned as well as scores
    if features:
        request_url = request_url+"?features=true"
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

### Query the score for each article based on revision ID

In [109]:
politicians_with_score = []
for politician in politician_with_revid:
    print("Getting ORES scores for '%s' with revid: %d"%(politician[0],politician[3]))
    score = request_ores_score_per_article(politician[3])
    scores = []
    for key in score['enwiki']['scores']:
        scores.append(key)
    pred = score['enwiki']['scores'][scores[0]]['articlequality']['score']['prediction']
    politician.append(pred)
    politicians_with_score.append(politician)

Getting ORES scores for 'Shahjahan Noori' with revid: 1099689043
Getting ORES scores for 'Abdul Ghafar Lakanwal' with revid: 943562276
Getting ORES scores for 'Majah Ha Adrif' with revid: 852404094
Getting ORES scores for 'Haroon al-Afghani' with revid: 1095102390
Getting ORES scores for 'Tayyab Agha' with revid: 1104998382
Getting ORES scores for 'Ahmadullah Wasiq' with revid: 1109361754
Getting ORES scores for 'Aziza Ahmadyar' with revid: 1087211008
Getting ORES scores for 'Muqadasa Ahmadzai' with revid: 1082489593
Getting ORES scores for 'Mohammad Sarwar Ahmedzai' with revid: 1038918070
Getting ORES scores for 'Amir Muhammad Akhundzada' with revid: 1069322182
Getting ORES scores for 'Nasrullah Baryalai Arsalai' with revid: 1095526840
Getting ORES scores for 'Mohammad Asim Asim' with revid: 1013838830
Getting ORES scores for 'Atiqullah Atifmal' with revid: 1112407669
Getting ORES scores for 'Abdul Rahim Ayoubi' with revid: 1108886061
Getting ORES scores for 'Alhaj Mutalib Baig' with 

Getting ORES scores for 'Bardhok Biba' with revid: 1110580984
Getting ORES scores for 'Dervish Biçaku' with revid: 1106794267
Getting ORES scores for 'Kostandin Boshnjaku' with revid: 1107876476
Getting ORES scores for 'Spartak Braho' with revid: 1098235893
Getting ORES scores for 'Safet Butka' with revid: 1067080697
Getting ORES scores for 'Uran Butka' with revid: 1087272778
Getting ORES scores for 'Refo Çapari' with revid: 1061419179
Getting ORES scores for 'Neritan Ceka' with revid: 1095178618
Getting ORES scores for 'Hysni Curri' with revid: 1114418989
Getting ORES scores for 'Ibrahim Dalliu' with revid: 1087026828
Getting ORES scores for 'Niazi Demi' with revid: 1104589367
Getting ORES scores for 'Abdurrahman Dibra' with revid: 1101365003
Getting ORES scores for 'Fuad Dibra' with revid: 1079550815
Getting ORES scores for 'Isuf Dibra' with revid: 1108975040
Getting ORES scores for 'Bujar Dida' with revid: 1112281152
Getting ORES scores for 'Ahmed Dino' with revid: 1114424802
Gettin

Getting ORES scores for 'André Luís Brandão' with revid: 1092527809
Getting ORES scores for 'André Gaspar Mendes de Carvalho' with revid: 871672690
Getting ORES scores for 'Carolina Cerqueira' with revid: 1081277226
Getting ORES scores for 'Arlete Chimbinda' with revid: 1101656146
Getting ORES scores for 'Kafundanga Chingunji' with revid: 1111470901
Getting ORES scores for 'Jeremias Chitunda' with revid: 1076646666
Getting ORES scores for 'Abel Chivukuvuku' with revid: 987997249
Getting ORES scores for 'Adalberto Costa Júnior' with revid: 1107102259
Getting ORES scores for 'Adjany Costa' with revid: 1081302172
Getting ORES scores for 'Álvaro Arnaldo Craveiro' with revid: 1112653739
Getting ORES scores for 'Raul Danda' with revid: 1099408586
Getting ORES scores for 'Carlos António Fernandes' with revid: 1003396569
Getting ORES scores for 'Fiel Domingos Constantino' with revid: 1082902869
Getting ORES scores for 'Abraão Gourgel' with revid: 1080440646
Getting ORES scores for 'Maria Monte

Getting ORES scores for 'Aram Grigoryan' with revid: 1112117000
Getting ORES scores for 'Armen Grigoryan (politician)' with revid: 1069944070
Getting ORES scores for 'Hayk Grigoryan' with revid: 1090623549
Getting ORES scores for 'Grigor Jaghetyan' with revid: 1062820993
Getting ORES scores for 'Tigran Karapetyan' with revid: 1112833785
Getting ORES scores for 'Khachatur-Bek of Mush' with revid: 1114420639
Getting ORES scores for 'Aram Kocharyan' with revid: 1107982644
Getting ORES scores for 'Davit Lokyan' with revid: 1089241404
Getting ORES scores for 'Hrant Maloyan' with revid: 1114902744
Getting ORES scores for 'Khachatur Malumian' with revid: 1114420669
Getting ORES scores for 'Aram Manukyan' with revid: 1048512708
Getting ORES scores for 'Gevorg Melik-Karagyozyan' with revid: 980519720
Getting ORES scores for 'Vahan Minakhoryan' with revid: 1055697655
Getting ORES scores for 'Ruben Mirzakhanyan' with revid: 1084737845
Getting ORES scores for 'Badal Muradyan' with revid: 109063053

Getting ORES scores for 'Fuad Alasgarov' with revid: 1018398706
Getting ORES scores for 'Anvar Alikhanov' with revid: 1109309028
Getting ORES scores for 'Tarlan Aliyarbayov' with revid: 1112064160
Getting ORES scores for 'Dilara Aliyeva' with revid: 1109649588
Getting ORES scores for 'Ali Asadov' with revid: 1112134949
Getting ORES scores for 'Akram Aylisli' with revid: 1106344373
Getting ORES scores for 'Sahil Babayev' with revid: 1112800095
Getting ORES scores for 'Huseyngulu Baghirov' with revid: 1080007779
Getting ORES scores for 'Alovsat Bakhshiyev' with revid: 1090036474
Getting ORES scores for 'Rafael Kamil Dzhabrailov' with revid: 1100266382
Getting ORES scores for 'Matin Eynullayev' with revid: 1110146340
Getting ORES scores for 'Vilayat Eyvazov' with revid: 1109686454
Getting ORES scores for 'Abdurrahman Fatalibeyli' with revid: 1094730548
Getting ORES scores for 'Fazail Ibrahimli' with revid: 1096310851
Getting ORES scores for 'Solomon Grobshtein' with revid: 1067081975
Gett

...

Getting ORES scores for 'Mounir Baatour' with revid: 1110930036
Getting ORES scores for 'Selma Baccar' with revid: 1044685883
Getting ORES scores for 'Yadh Ben Achour' with revid: 1101801981
Getting ORES scores for 'Ahmed Ben Salah' with revid: 1061990220
Getting ORES scores for 'Leila Ben-Gacem' with revid: 1106842255
Getting ORES scores for 'Mohamed Nejib Berriche' with revid: 1100422221
Getting ORES scores for 'Ghazi Chaouachi' with revid: 1094076051
Getting ORES scores for 'Ali Chaouch' with revid: 1101144527
Getting ORES scores for 'Mohamed Charfi' with revid: 1098183988
Getting ORES scores for 'Mohamed Messaoud Chebbi' with revid: 1105924035
Getting ORES scores for 'Ahmad ibn Abi Diyaf' with revid: 1111867713
Getting ORES scores for 'Mahmoud Djellouli' with revid: 1114898394
Getting ORES scores for 'Mohamed Salah Fliss' with revid: 1021846452
Getting ORES scores for 'Mohamed Ghannouchi' with revid: 1102676192
Getting ORES scores for 'Rached Ghannouchi' with revid: 1109470504
Gett

Getting ORES scores for 'Florence Akiiki Asiimwe' with revid: 1027457516
Getting ORES scores for 'Charles Ayume' with revid: 1086952268
Getting ORES scores for 'Edith Mary Bataringaya' with revid: 1095198944
Getting ORES scores for 'Flavia Rwabuhoro Kabahenda' with revid: 1081460224
Getting ORES scores for 'John Kabaireho' with revid: 944462983
Getting ORES scores for 'Joseph Kabuleta' with revid: 1112443092
Getting ORES scores for 'Apollo Kaggwa' with revid: 1074999358
Getting ORES scores for 'Michael Kawalya Kagwa' with revid: 943104033
Getting ORES scores for 'Robert Mugabe Kakyebezi' with revid: 1055443363
Getting ORES scores for 'Paulo Kavuma' with revid: 1079821209
Getting ORES scores for 'Patrick Kayemba' with revid: 1036632663
Getting ORES scores for 'Daniel Kidega' with revid: 1110604657
Getting ORES scores for 'Yabezi Kiiza' with revid: 994379167
Getting ORES scores for 'Pumla Kisosonkole' with revid: 1092915114
Getting ORES scores for 'Annie Logiel' with revid: 1078602113
Ge

Getting ORES scores for 'Hamdan bin Rashid Al Maktoum' with revid: 1110689861
Getting ORES scores for 'Maktoum bin Rashid Al Maktoum' with revid: 1107670119
Getting ORES scores for 'Mohammed bin Khalifa Al Maktoum' with revid: 949517140
Getting ORES scores for 'Mohammed bin Rashid Al Maktoum' with revid: 1115862687
Getting ORES scores for 'Saeed bin Maktoum bin Hasher Al Maktoum' with revid: 1102931394
Getting ORES scores for 'Hawaa Al Mansoori' with revid: 1110913016
Getting ORES scores for 'Sultan bin Saeed Al Mansoori' with revid: 1092057370
Getting ORES scores for 'Mansour bin Zayed Al Nahyan' with revid: 1114026346
Getting ORES scores for 'Abdulla bin Touq Al Marri' with revid: 1104185628
Getting ORES scores for 'Mohammed bin Saud Al Qasimi' with revid: 1096731396
Getting ORES scores for 'Abdullah bin Zayed Al Nahyan' with revid: 1105633932
Getting ORES scores for 'Hamdan bin Zayed bin Sultan Al Nahyan' with revid: 1103862154
Getting ORES scores for 'Hamed bin Zayed Al Nahyan' wit

Getting ORES scores for 'Nelson Martínez (politician)' with revid: 1107146847
Getting ORES scores for 'Carlos Mata Figueroa' with revid: 1106249914
Getting ORES scores for 'Manuel Antonio Matos' with revid: 1073559994
Getting ORES scores for 'Ricardo José Menéndez Prieto' with revid: 1059606624
Getting ORES scores for 'Nelson Merentes' with revid: 1070050745
Getting ORES scores for 'Francisco de Miranda' with revid: 1112724061
Getting ORES scores for 'Diego Alfredo Molero Bellavia' with revid: 1012172289
Getting ORES scores for 'Venancio Antonio Morin' with revid: 959659970
Getting ORES scores for 'Reinaldo Muñoz Pedroza' with revid: 1111138140
Getting ORES scores for 'Dia Nader de El-Andari' with revid: 1084711403
Getting ORES scores for 'Moisés Naím' with revid: 1100729862
Getting ORES scores for 'Freddy Ñáñez' with revid: 1107284988
Getting ORES scores for 'Ana Elisa Osorio' with revid: 1083403033
Getting ORES scores for 'Renny Ottolina' with revid: 1102102358
Getting ORES scores fo

Getting ORES scores for 'Mirriam Chinyama Chonya' with revid: 1090872334
Getting ORES scores for 'Rosaria C. Fundanga' with revid: 1090872335
Getting ORES scores for 'Chibesa Kankasa' with revid: 1090872459
Getting ORES scores for 'Chushi C. Kasanda' with revid: 1090872343
Getting ORES scores for 'Mwamba Luchembe' with revid: 1105621834
Getting ORES scores for 'Muyoba Macwani' with revid: 1056038831
Getting ORES scores for 'Kampamba Mulenga Chilumba' with revid: 1049942660
Getting ORES scores for 'Mike Mulongoti' with revid: 1110802508
Getting ORES scores for 'Peter Sinkamba' with revid: 1105649973
Getting ORES scores for 'Yobert K. Shamapande' with revid: 1002399039
Getting ORES scores for 'Brenda Mwika Tambatamba' with revid: 1090872349
Getting ORES scores for 'Gift Banda' with revid: 1062652620
Getting ORES scores for 'Flora Buka' with revid: 1084849963
Getting ORES scores for 'Joseph Makamba Busha' with revid: 1021924531
Getting ORES scores for 'James Chikerema' with revid: 9187863

In [110]:
politicians_with_score

[['Shahjahan Noori',
  'https://en.wikipedia.org/wiki/Shahjahan_Noori',
  'Afghanistan',
  1099689043,
  1099689043,
  1099689043,
  'GA'],
 ['Abdul Ghafar Lakanwal',
  'https://en.wikipedia.org/wiki/Abdul_Ghafar_Lakanwal',
  'Afghanistan',
  943562276,
  943562276,
  943562276,
  'Start'],
 ['Majah Ha Adrif',
  'https://en.wikipedia.org/wiki/Majah_Ha_Adrif',
  'Afghanistan',
  852404094,
  852404094,
  852404094,
  'Start'],
 ['Haroon al-Afghani',
  'https://en.wikipedia.org/wiki/Haroon_al-Afghani',
  'Afghanistan',
  1095102390,
  1095102390,
  1095102390,
  'B'],
 ['Tayyab Agha',
  'https://en.wikipedia.org/wiki/Tayyab_Agha',
  'Afghanistan',
  1104998382,
  1104998382,
  1104998382,
  'Start'],
 ['Ahmadullah Wasiq',
  'https://en.wikipedia.org/wiki/Ahmadullah_Wasiq',
  'Afghanistan',
  1109361754,
  1109361754,
  1109361754,
  'Start'],
 ['Aziza Ahmadyar',
  'https://en.wikipedia.org/wiki/Aziza_Ahmadyar',
  'Afghanistan',
  1087211008,
  1087211008,
  1087211008,
  'Start'],
 ['Muq

In [111]:
regions = {
    "NORTHERN AFRICA": [],
    "WESTERN AFRICA": [],
    "EASTERN AFRICA": [],
    "MIDDLE AFRICA": [],
    "SOUTHERN AFRICA": [],
    
    "NORTHERN AMERICA": [],
    
    "CENTRAL AMERICA": [],
    "CARIBBEAN": [],
    "SOUTH AMERICA": [],
    
    "WESTERN ASIA": [],
    "CENTRAL ASIA": [],
    "SOUTH ASIA": [],
    "SOUTHEAST ASIA": [],
    "EAST ASIA": [],
    
    "NORTHERN EUROPE": [],
    "WESTERN EUROPE": [],
    "EASTERN EUROPE": [],
    "SOUTHERN EUROPE": [],
    
    "OCEANIA": []
}

with open('population_by_country_2022.csv', encoding='utf-8', mode ='r') as f:
    reader = csv.reader(f, delimiter=',')
    curr = list(regions)[0]
    next = list(regions)[1]
    i = 1
    for row in reader:
        if row[0] == 'Geography':
            print(row)
        elif not row[0].isupper():
            regions[curr].append(row)
        elif row[0] == next:
            curr = next
            i = i+1
            if i < len(list(regions)):
                next = list(regions)[i]
regions

['Geography', 'Population (millions)']


{'NORTHERN AFRICA': [['Algeria', '44.9'],
  ['Egypt', '103.5'],
  ['Libya', '6.8'],
  ['Morocco', '36.7'],
  ['Sudan', '46.9'],
  ['Tunisia', '11.8'],
  ['Western Sahara', '0.6']],
 'WESTERN AFRICA': [['Benin', '13.4'],
  ['Burkina Faso', '22.7'],
  ['Cape Verde', '0.6'],
  ["Cote d'Ivoire", '28.2'],
  ['Gambia', '2.7'],
  ['Ghana', '33.5'],
  ['Guinea', '13.9'],
  ['Guinea-Bissau', '2.1'],
  ['Liberia', '5.3'],
  ['Mali', '22.6'],
  ['Mauritania', '4.7'],
  ['Niger', '26.2'],
  ['Nigeria', '218.5'],
  ['Senegal', '17.9'],
  ['Sierra Leone', '8.7'],
  ['Togo', '8.8']],
 'EASTERN AFRICA': [['Burundi', '12.9'],
  ['Comoros', '0.8'],
  ['Djibouti', '1.1'],
  ['Eritrea', '3.7'],
  ['Ethiopia', '123.4'],
  ['Kenya', '54'],
  ['Madagascar', '29.6'],
  ['Malawi', '20.4'],
  ['Mauritius', '1.3'],
  ['Mayotte', '0.3'],
  ['Mozambique', '33'],
  ['Reunion', '0.9'],
  ['Rwanda', '13.8'],
  ['Seychelles', '0.1'],
  ['Somalia', '17.6'],
  ['South Sudan', '10.9'],
  ['Tanzania', '65.5'],
  ['Uganda'

### Attach country/region info to each Politician's article record
Looks like there is no match found for Korean articles.

In [138]:
no_match = []
wp_politicians_by_country = []
for politician in politicians_with_score:
    country = politician[2]
    match = 0
    for region in regions:
        for c in regions[region]:
            if c[0] == country:
                match = 1
                p = []
                p.append(country)
                p.append(region)
                p.append(c[1])
                p.append(politician[0])
                p.append(politician[3])
                #p.append(politician[4])
                p.append(politician[-1])
                wp_politicians_by_country.append(p)
    if match == 0:
        print(country)

Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean
Korean


In [176]:
wp_politicians_by_country

[['Afghanistan', 'SOUTH ASIA', '41.1', 'Shahjahan Noori', 1099689043, 'GA'],
 ['Afghanistan',
  'SOUTH ASIA',
  '41.1',
  'Abdul Ghafar Lakanwal',
  943562276,
  'Start'],
 ['Afghanistan', 'SOUTH ASIA', '41.1', 'Majah Ha Adrif', 852404094, 'Start'],
 ['Afghanistan', 'SOUTH ASIA', '41.1', 'Haroon al-Afghani', 1095102390, 'B'],
 ['Afghanistan', 'SOUTH ASIA', '41.1', 'Tayyab Agha', 1104998382, 'Start'],
 ['Afghanistan',
  'SOUTH ASIA',
  '41.1',
  'Ahmadullah Wasiq',
  1109361754,
  'Start'],
 ['Afghanistan', 'SOUTH ASIA', '41.1', 'Aziza Ahmadyar', 1087211008, 'Start'],
 ['Afghanistan',
  'SOUTH ASIA',
  '41.1',
  'Muqadasa Ahmadzai',
  1082489593,
  'Start'],
 ['Afghanistan',
  'SOUTH ASIA',
  '41.1',
  'Mohammad Sarwar Ahmedzai',
  1038918070,
  'Start'],
 ['Afghanistan',
  'SOUTH ASIA',
  '41.1',
  'Amir Muhammad Akhundzada',
  1069322182,
  'Start'],
 ['Afghanistan',
  'SOUTH ASIA',
  '41.1',
  'Nasrullah Baryalai Arsalai',
  1095526840,
  'Start'],
 ['Afghanistan',
  'SOUTH ASIA',
  

### Consolidate the data in a single file `wp_politicians_by_country.csv`

In [180]:
pandas.DataFrame(
    wp_politicians_by_country,
    columns = ['country','region','population','article_title','revision_id','article_quality']
).to_csv("wp_politicians_by_country.csv", index = False)

### Coverage computation
For each country, compute the total number of articles, total number of high quality articles, and the subsequent coverage per capita.

In [158]:
countries_coverage = []
country = wp_politicians_by_country[0][0]
region = wp_politicians_by_country[0][1]
population = wp_politicians_by_country[0][2]
articles = 0
high_quality = 0
low_quality = 0
for p in wp_politicians_by_country:
    if p[0] == country:
        articles = articles + 1
        if p[-1] == 'FA':
            high_quality = high_quality + 1
        elif p[-1] == 'GA':
            high_quality = high_quality + 1
        else:
            low_quality = low_quality + 1
    elif float(population) == 0:
        print(country)
        country = p[0]
        region = p[1]
        population = p[2]
        articles = 0
        high_quality = 0
        low_quality = 0
    else:
        c = []
        c.append(country)
        c.append(region)
        c.append(population)
        c.append(articles)
        coverage = articles / (float(population)*1000000)
        c.append(coverage)
        c.append(high_quality)
        coverage_high = high_quality / (float(population)*1000000)
        c.append(coverage_high)
        #c.append(low_quality)
        #coverage_low = low_quality / (float(population)*1000000)
        #c.append(coverage_low)
        countries_coverage.append(c)
        country = p[0]
        region = p[1]
        population = p[2]
        articles = 0
        high_quality = 0
        low_quality = 0
countries_coverage

Liechtenstein
Monaco
Nauru
Palau
San Marino
Tuvalu


[['Afghanistan',
  'SOUTH ASIA',
  '41.1',
  118,
  2.871046228710462e-06,
  6,
  1.4598540145985402e-07],
 ['Albania',
  'SOUTHERN EUROPE',
  '2.8',
  82,
  2.9285714285714284e-05,
  5,
  1.7857142857142857e-06],
 ['Algeria', 'NORTHERN AFRICA', '44.9', 33, 7.34966592427617e-07, 0, 0.0],
 ['Andorra', 'SOUTHERN EUROPE', '0.1', 9, 9e-05, 1, 1e-05],
 ['Angola', 'MIDDLE AFRICA', '35.6', 41, 1.151685393258427e-06, 0, 0.0],
 ['Antigua and Barbuda', 'CARIBBEAN', '0.1', 16, 0.00016, 0, 0.0],
 ['Argentina', 'SOUTH AMERICA', '46.2', 53, 1.1471861471861472e-06, 0, 0.0],
 ['Armenia', 'WESTERN ASIA', '3', 45, 1.5e-05, 1, 3.3333333333333335e-07],
 ['Austria', 'WESTERN EUROPE', '9', 85, 9.444444444444445e-06, 0, 0.0],
 ['Azerbaijan',
  'WESTERN ASIA',
  '10.2',
  55,
  5.392156862745098e-06,
  1,
  9.80392156862745e-08],
 ['Bahamas', 'CARIBBEAN', '0.4', 2, 5e-06, 0, 0.0],
 ['Bahrain', 'WESTERN ASIA', '1.5', 35, 2.3333333333333332e-05, 0, 0.0],
 ['Bangladesh', 'SOUTH ASIA', '171.2', 55, 3.212616822429

### Results
The results are shown in the following 6 data tables.

### 1. Top 10 countries by coverage:
The 10 countries with the highest total articles per capita (in descending order) 

In [172]:
import pandas
coverage = pandas.DataFrame(countries_coverage, columns=['Country', 'Region',
                                     'Population', 'Articles', 'Coverage', 'High', 'Coverage High'])
coverage_desc = coverage.sort_values(by=['Coverage'], ascending=False)
coverage_desc.head(10).set_index('Country')

,Region,Population,Articles,Coverage,High,Coverage High
Country,,,,,,
Antigua and Barbuda,CARIBBEAN,0.1,16,0.000160,0,0.000000
Federated States of Micronesia,OCEANIA,0.1,12,0.000120,0,0.000000
Andorra,SOUTHERN EUROPE,0.1,9,0.000090,1,0.000010
Barbados,CARIBBEAN,0.3,27,0.000090,0,0.000000
Marshall Islands,OCEANIA,0.1,8,0.000080,0,0.000000
Montenegro,SOUTHERN EUROPE,0.6,35,0.000058,2,0.000003
Luxembourg,WESTERN EUROPE,0.7,36,0.000051,0,0.000000
Bhutan,SOUTH ASIA,0.8,40,0.000050,0,0.000000
Seychelles,EASTERN AFRICA,0.1,5,0.000050,0,0.000000


### 2. Bottom 10 countries by coverage:
The 10 countries with the lowest total articles per capita (in ascending order)

In [162]:
coverage_asc = coverage.sort_values(by=['Coverage'], ascending=True)
coverage_asc.head(10).set_index('Country')

,Region,Population,Articles,Coverage,High,Coverage High
Country,,,,,,
Saint Lucia,CARIBBEAN,0.2,0,0.000000e+00,0,0.000000e+00
Mexico,CENTRAL AMERICA,127.5,0,0.000000e+00,0,0.000000e+00
China,EAST ASIA,1436.6,1,6.960880e-10,0,0.000000e+00
Romania,EASTERN EUROPE,19,1,5.263158e-08,1,5.263158e-08
Saudi Arabia,WESTERN ASIA,36.7,2,5.449591e-08,1,2.724796e-08
Sri Lanka,SOUTH ASIA,22.4,2,8.928571e-08,0,0.000000e+00
India,SOUTH ASIA,1417.2,177,1.248942e-07,6,4.233700e-09
Egypt,NORTHERN AFRICA,103.5,13,1.256039e-07,0,0.000000e+00
Taiwan,EAST ASIA,23.2,4,1.724138e-07,0,0.000000e+00


### 3. Top 10 countries by high quality:
The 10 countries with the highest high quality articles per capita (in descending order)

In [164]:
coverage_high_desc = coverage.sort_values(by=['Coverage High'], ascending=False)
coverage_high_desc.head(10).set_index('Country')

,Region,Population,Articles,Coverage,High,Coverage High
Country,,,,,,
Andorra,SOUTHERN EUROPE,0.1,9,0.000090,1,1.000000e-05
Montenegro,SOUTHERN EUROPE,0.6,35,0.000058,2,3.333333e-06
Albania,SOUTHERN EUROPE,2.8,82,0.000029,5,1.785714e-06
Suriname,SOUTH AMERICA,0.6,22,0.000037,1,1.666667e-06
Bosnia-Herzegovina,SOUTHERN EUROPE,3.4,51,0.000015,5,1.470588e-06
Lithuania,NORTHERN EUROPE,2.8,74,0.000026,3,1.071429e-06
Croatia,SOUTHERN EUROPE,3.8,53,0.000014,4,1.052632e-06
Slovenia,SOUTHERN EUROPE,2.1,42,0.000020,2,9.523810e-07
Palestinian Territory,WESTERN ASIA,5.4,70,0.000013,5,9.259259e-07


### 4. Bottom 10 countries by high quality:
The 10 countries with the lowest high quality articles per capita (in ascending order)

In [165]:
coverage_high_asc = coverage.sort_values(by=['Coverage High'], ascending=True)
coverage_high_asc.head(10).set_index('Country')

,Region,Population,Articles,Coverage,High,Coverage High
Country,,,,,,
Zambia,EASTERN AFRICA,20,12,6.000000e-07,0,0.0
Laos,SOUTHEAST ASIA,7.5,5,6.666667e-07,0,0.0
Kuwait,WESTERN ASIA,4.1,10,2.439024e-06,0,0.0
Sri Lanka,SOUTH ASIA,22.4,2,8.928571e-08,0,0.0
St. Kitts-Nevis,CARIBBEAN,0.1,2,2.000000e-05,0,0.0
Jamaica,CARIBBEAN,2.8,18,6.428571e-06,0,0.0
Italy,SOUTHERN EUROPE,58.9,156,2.648557e-06,0,0.0
Israel,WESTERN ASIA,9.5,4,4.210526e-07,0,0.0
Iceland,NORTHERN EUROPE,0.4,12,3.000000e-05,0,0.0


### 5. Geographic regions by total coverage:
A rank ordered list of geographic regions (in descending order) by total articles per capita

In [169]:
region_coverage = coverage.groupby(["Region"]).sum()
region_coverage_desc = region_coverage.sort_values(by=['Coverage'], ascending=False)
region_coverage_desc

,Articles,Coverage,High,Coverage High
Region,,,,
CARIBBEAN,188,0.000391,8,7.144244e-07
SOUTHERN EUROPE,874,0.000300,42,2.027017e-05
OCEANIA,64,0.000294,1,1.075269e-07
EASTERN AFRICA,571,0.000112,15,7.364417e-07
WESTERN ASIA,668,0.000104,27,3.022542e-06
NORTHERN EUROPE,254,0.000099,8,2.311559e-06
SOUTH ASIA,640,0.000091,23,3.234467e-07
SOUTH AMERICA,565,0.000086,12,2.157234e-06
WESTERN EUROPE,677,0.000082,22,8.118000e-07


### 6. Geographic regions by high quality coverage:
Rank ordered list of geographic regions (in descending order) by high quality articles per capita

In [170]:
region_coverage_high_desc = region_coverage.sort_values(by=['Coverage High'], ascending=False)
region_coverage_high_desc

,Articles,Coverage,High,Coverage High
Region,,,,
SOUTHERN EUROPE,874,0.000300,42,2.027017e-05
WESTERN ASIA,668,0.000104,27,3.022542e-06
NORTHERN EUROPE,254,0.000099,8,2.311559e-06
SOUTH AMERICA,565,0.000086,12,2.157234e-06
EASTERN EUROPE,725,0.000046,37,1.577002e-06
CENTRAL AMERICA,187,0.000033,10,1.416305e-06
MIDDLE AFRICA,195,0.000020,4,1.047747e-06
WESTERN EUROPE,677,0.000082,22,8.118000e-07
EASTERN AFRICA,571,0.000112,15,7.364417e-07
